In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('DA_Test').config("spark.jars.packages", "com.crealytics:spark-excel_2.12:0.13.7").getOrCreate()
from pyspark.sql import *
import pyspark.sql.functions as F
from pyspark.sql.functions import col, regexp_replace, date_format, to_timestamp, expr,from_unixtime,unix_timestamp,concat_ws, when

In [7]:
df = spark.read \
    .format("com.crealytics.spark.excel") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("data/Python_exercise1.xlsx")
    

In [11]:
df.show(1,False)

+---+---------------------+------+---------------+-----+-----------+-----------+-------+----------+--------------+------+-----------------+-------------------+----------------+--------------+------------------+----------------+------+----------------+--------+------+----------+--------+-------+-------+----------------+---------------+------------+------------+---------+---------+------------+-------+---------+--------+---------+------+---------+----------+----------+-----+----+-----------+----------------+----------------+---------------------+--------------------------------------------------------------+-------------------------------+
|No |Time                 |Level |Firewall Action|User |Source     |Destination|Service|Sent Bytes|Received Bytes|Threat|Application      |Application Details|Application Type|Application ID|Application Action|Application List|Attack|Device ID       |Duration|Log ID|Session ID|Sub Type|Type   |Message|Destination Name|Security Action|UTM Severity|UTM S

In [107]:
from pyspark.sql.functions import to_date, to_timestamp,trim,regexp_extract

In [23]:
df = df.withColumn('Time_temp', regexp_replace(col('Time'), '\.', ':'))

In [42]:
df.dtypes

[('No', 'double'),
 ('Time', 'string'),
 ('Level', 'string'),
 ('Firewall Action', 'string'),
 ('User', 'string'),
 ('Source', 'string'),
 ('Destination', 'string'),
 ('Service', 'string'),
 ('Sent Bytes', 'double'),
 ('Received Bytes', 'double'),
 ('Threat', 'string'),
 ('Application', 'string'),
 ('Application Details', 'string'),
 ('Application Type', 'string'),
 ('Application ID', 'double'),
 ('Application Action', 'string'),
 ('Application List', 'string'),
 ('Attack', 'string'),
 ('Device ID', 'string'),
 ('Duration', 'double'),
 ('Log ID', 'double'),
 ('Session ID', 'double'),
 ('Sub Type', 'string'),
 ('Type', 'string'),
 ('Message', 'string'),
 ('Destination Name', 'string'),
 ('Security Action', 'string'),
 ('UTM Severity', 'string'),
 ('UTM Sub Type', 'string'),
 ('UTM Event', 'string'),
 ('Policy ID', 'double'),
 ('Tran Display', 'string'),
 ('Tran IP', 'string'),
 ('Tran Port', 'double'),
 ('Protocol', 'double'),
 ('File Name', 'string'),
 ('Sender', 'string'),
 ('Recipien

In [66]:
from pyspark.sql.functions import regexp_replace, concat, lpad, col,lit
#from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, concat, lpad, col,lit,split,size
from pyspark.sql.types import StringType

# Initialize Spark session
#spark = SparkSession.builder.appName("DateConversionWithRegex").getOrCreate()

# Extract components using regexp_extract
df = df.withColumn("date", split(df["Time_temp"], " ")[0])
df = df.withColumn("time", split(df["Time_temp"], "  ")[1])

# Extract date components
df = df.withColumn("day", split(df["date"], "/")[0].cast("int"))
df = df.withColumn("month", split(df["date"], "/")[1].cast("int"))
df = df.withColumn("year", split(df["date"], "/")[2].cast("int"))



df = df.withColumn("hour", split(df["time"], ":")[0])

df = df.withColumn("minute", split(df["time"], ":")[1])

df = df.withColumn("second", split(df["time"], ":")[2].substr(1,2))


# Show the results
df.select("Time_temp", "day","month","year","hour","minute","second").show(truncate=False)


+---------------------+---+-----+----+----+------+------+
|Time_temp            |day|month|year|hour|minute|second|
+---------------------+---+-----+----+----+------+------+
|1/8/2022  10:01:00 AM|1  |8    |2022|10  |01    |00    |
|1/8/2022  10:01:00 AM|1  |8    |2022|10  |01    |00    |
|1/8/2022  10:01:00 AM|1  |8    |2022|10  |01    |00    |
|1/8/2022  10:01:00 AM|1  |8    |2022|10  |01    |00    |
|1/8/2022  10:01:00 AM|1  |8    |2022|10  |01    |00    |
|1/8/2022  10:01:00 AM|1  |8    |2022|10  |01    |00    |
|1/8/2022  10:01:00 AM|1  |8    |2022|10  |01    |00    |
|1/8/2022  10:01:00 AM|1  |8    |2022|10  |01    |00    |
|1/8/2022  10:01:00 AM|1  |8    |2022|10  |01    |00    |
|1/8/2022  10:01:00 AM|1  |8    |2022|10  |01    |00    |
|1/8/2022  10:01:00 AM|1  |8    |2022|10  |01    |00    |
|1/8/2022  10:01:00 AM|1  |8    |2022|10  |01    |00    |
|1/8/2022  10:01:00 AM|1  |8    |2022|10  |01    |00    |
|1/8/2022  10:01:00 AM|1  |8    |2022|10  |01    |00    |
|1/8/2022  10:

In [67]:
from pyspark.sql.functions import format_string
df = df.withColumn("d1", format_string("%02d", col("month")) )
df = df.withColumn("d2", format_string("%02d", col("day")) )
df = df.withColumn("time", concat_ws("/", df["hour"], df["minute"], df["second"]))
df =df.withColumn("date", concat_ws("/" , df["d2"], df["d1"], df["year"]))

df =df.withColumn("time_new", concat_ws(" ",df["time"],df["date"]))
#Assignment Q1 answer 
df.select("time_new").distinct().show(truncate=False)

+-------------------+
|time_new           |
+-------------------+
|10/01/00 01/08/2022|
|10/03/00 01/08/2022|
|10/04/00 01/08/2022|
|10/07/00 01/08/2022|
|10/02/00 01/08/2022|
|10/06/00 01/08/2022|
|10/05/00 01/08/2022|
|10/09/00 01/08/2022|
|10/13/00 01/08/2022|
|10/08/00 01/08/2022|
|10/12/00 01/08/2022|
|10/11/00 01/08/2022|
|10/10/00 01/08/2022|
|10/19/00 01/08/2022|
|10/15/00 01/08/2022|
|10/17/00 01/08/2022|
|10/14/00 01/08/2022|
|10/16/00 01/08/2022|
|10/18/00 01/08/2022|
|10/25/00 01/08/2022|
+-------------------+
only showing top 20 rows



In [ ]:
#) Find and display the different values in the column "Level", "Firewall", "User", "Service" and its 
# distribution

In [71]:
df.select("Level","Firewall Action","User","Service").distinct().show(10,False)

+------+---------------+-------+--------+
|Level |Firewall Action|User   |Service |
+------+---------------+-------+--------+
|notice|accept         |user2  |NTP     |
|notice|client-rst     |pandora|HTTP    |
|notice|timeout        |user1  |tcp/9100|
|notice|close          |guest  |HTTP    |
|notice|close          |pandora|HTTPS   |
|notice|accept         |user2  |SNMP    |
|notice|accept         |user1  |DNS     |
|notice|client-rst     |guest  |HTTPS   |
|notice|server-rst     |guest  |HTTP    |
|notice|client-rst     |guest  |HTTP    |
+------+---------------+-------+--------+
only showing top 10 rows



In [75]:
df.groupby("Level").count().show(10,False)

+------+-----+
|Level |count|
+------+-----+
|notice|7596 |
|ignore|2914 |
+------+-----+



In [76]:
df.groupby("Firewall Action").count().show(10,False)

+---------------+-----+
|Firewall Action|count|
+---------------+-----+
|accept         |5223 |
|timeout        |97   |
|close          |3329 |
|server-rst     |636  |
|client-rst     |1223 |
|deny           |2    |
+---------------+-----+



In [79]:
df.groupby("Service").count().show(200,False)

+------------------------+-----+
|Service                 |count|
+------------------------+-----+
|DNS                     |4998 |
|udp/137                 |16   |
|NTP                     |28   |
|SNMP                    |51   |
|udp/443                 |88   |
|HTTPS                   |3815 |
|tcp/9100                |76   |
|HTTP                    |1180 |
|KERBEROS                |54   |
|LDAP_UDP                |7    |
|Internet-Locator-Service|32   |
|RDP                     |24   |
|SMB                     |13   |
|PING                    |30   |
|tcp/7680                |4    |
|tcp/49666               |18   |
|DCE-RPC                 |35   |
|udp/3544                |3    |
|IMAPS                   |8    |
|tcp/49672               |21   |
|tcp/8080                |5    |
|udp/46597               |1    |
|udp/3389                |3    |
+------------------------+-----+



In [81]:
df.groupby("User").count().show(10,False)

+--------------+-----+
|User          |count|
+--------------+-----+
|user1         |3757 |
|user2         |586  |
|guest         |5980 |
|pandora       |112  |
|aiswaryasuresh|75   |
+--------------+-----+



 Based on the column Service, if it is HTTPS and the Sent Bytes or Received Bytes are greater than 
 20000 set a flag as "a_000x8". If Service has any other value and the Sent Bytes or Received Bytes are 
 greater than 10000 set a flag as "v_000x2" 



In [84]:
df.select("Service","Sent Bytes","Received Bytes").distinct().show(10,False)

+-------+----------+--------------+
|Service|Sent Bytes|Received Bytes|
+-------+----------+--------------+
|HTTPS  |8956.0    |3068.0        |
|DNS    |57.0      |73.0          |
|HTTPS  |662.0     |662.0         |
|DNS    |85.0      |238.0         |
|HTTPS  |4126.0    |52292.0       |
|DNS    |148.0     |262.0         |
|HTTPS  |2092.0    |9535.0        |
|HTTPS  |3292.0    |43940.0       |
|HTTPS  |3202.0    |50023.0       |
|HTTPS  |3326.0    |8208.0        |
+-------+----------+--------------+
only showing top 10 rows



In [92]:
df = df.withColumn("flag", expr("""
    CASE 
        WHEN Service = 'HTTPS' OR `Sent Bytes` > 20000 OR `Received Bytes` > 20000 THEN 'a_000x8' 
        WHEN `Sent Bytes` > 10000 OR `Received Bytes` > 10000 THEN 'v_000x2' 
        ELSE NULL 
    END
"""))

In [93]:
df.select("flag").distinct().show(10,False)

+-------+
|flag   |
+-------+
|null   |
|a_000x8|
|v_000x2|
+-------+



How manu unqiue devices are there in the dataset?

In [100]:
 df.select("Device ID").distinct().count()

4

The field ReportToManager has CN="XXXXXXX" which describes the user's manager. Find the manager 
to whom most users report to.

In [116]:
# df.dtypes

In [104]:
df.select("ReporttoManager").distinct().show(10,False)

+--------------------------------------------------------------+
|ReporttoManager                                               |
+--------------------------------------------------------------+
|CN=d921478,OU=People,OU=eProfile,DC=core,DC=dir,DC=tdog,DC=com|
|CN=d921479,OU=People,OU=eProfile,DC=core,DC=dir,DC=tdog,DC=com|
|CN=d921477,OU=People,OU=eProfile,DC=core,DC=dir,DC=tdog,DC=com|
+--------------------------------------------------------------+



In [108]:
df = df.withColumn("CN", regexp_extract("ReporttoManager", r"CN=([^,]+)", 1))
df.select("CN", "ReporttoManager").distinct().show(10,False)

+-------+--------------------------------------------------------------+
|CN     |ReporttoManager                                               |
+-------+--------------------------------------------------------------+
|d921478|CN=d921478,OU=People,OU=eProfile,DC=core,DC=dir,DC=tdog,DC=com|
|d921479|CN=d921479,OU=People,OU=eProfile,DC=core,DC=dir,DC=tdog,DC=com|
|d921477|CN=d921477,OU=People,OU=eProfile,DC=core,DC=dir,DC=tdog,DC=com|
+-------+--------------------------------------------------------------+



In [114]:
df.groupby("CN").count().orderBy(col("count").desc()).show(10,False)

+-------+-----+
|CN     |count|
+-------+-----+
|d921478|5566 |
|d921477|4912 |
|d921479|32   |
+-------+-----+



 Extract the fist component from the field enName and store is as a new column named enName1


In [118]:
df = df.withColumn("enName1",split(df["enName"],"\.")[0])

In [119]:
df.select("enName","enName1").distinct().show(10,False)

+-------------------------------+----------+
|enName                         |enName1   |
+-------------------------------+----------+
|wsaddc0007.core.dir.company.com|wsaddc0007|
|wsaddc0005.core.dir.company.com|wsaddc0005|
|wsaddc0002.core.dir.company.com|wsaddc0002|
|wsaddc0001.core.dir.company.com|wsaddc0001|
+-------------------------------+----------+

